### Using the GetOldTweets package to fetch tweets

In [ ]:
tweetCriteria = got.manager.TweetCriteria().setUsername("barackobama whitehouse")\
                                           .setMaxTweets(2)
tweet = got.manager.TweetManager.getTweets(tweetCriteria)[0]
print(tweet.text)

In [ ]:
tweetCriteria = got.manager.TweetCriteria().setUsername("jxndu")\
                                           .setMaxTweets(2)
tweet = got.manager.TweetManager.getTweets(tweetCriteria)[0]
print(tweet.text)

In [ ]:
tweetCriteria = got.manager.TweetCriteria().setUsername("jxndu")\
                                           .setSince("2015-09-10")\
                                           .setUntil("2016-01-01")\
                                           .setMaxTweets(10)\
                                           .setEmoji("unicode")
tweet = got.manager.TweetManager.getTweets(tweetCriteria)[6]
print(tweet.text)

In [ ]:
tweetCriteria = got.manager.TweetCriteria().setUsername("jxndu")\
                                           .setTopTweets(True)\
                                           .setMaxTweets(10)
tweet = got.manager.TweetManager.getTweets(tweetCriteria)[0]
print(tweet.text)

In [ ]:
#%load GetOldTweets3.py
#!/usr/bin/env python3
"""To use this script you can pass the following attributes:
       querysearch: a query text to be matched
          username: a username or a list of usernames (comma or space separated)
                    of a specific twitter account(s) (with or without @)
username-from-file: a file with a list of usernames,
             since: a lower bound date in UTC (yyyy-mm-dd)
             until: an upper bound date in UTC (yyyy-mm-dd) (not included)
              near: a reference location area from where tweets were generated
            within: a distance radius from "near" location (e.g. 15mi)
         toptweets: only the tweets provided as top tweets by Twitter (no parameters required)
         maxtweets: the maximum number of tweets to retrieve
              lang: the language of tweets
            output: a filename to export the results (default is "output_got.csv")

Examples:

# Example 1 - Get tweets by query search:
GetOldTweets3 --querysearch "europe refugees" --maxtweets 10

# Example 1 - Get the last 10 top tweets by username:
GetOldTweets3 --username "barackobama" --toptweets --maxtweets 10

# Example 3 - Get tweets by username and bound dates (until date is not included):
GetOldTweets3 --username "barackobama" --since 2015-09-10 --until 2015-09-12 --maxtweets 10

# Example 4 - Get tweets by several usernames:
GetOldTweets3 --username "BarackObama,AngelaMerkeICDU" --usernames-from-file userlist.txt --maxtweets 10

# Example 5 - Get tweets by language:
GetOldTweets3 --querysearch "bitcoin" --lang cn --maxtweets 10 

# Example 6 - Get tweets by place:
GetOldTweets3 --querysearch "bitcoin" --near "Berlin, Germany" --within 25km --maxtweets 10 

# Example 7 - Get tweets by geo coordinates:
GetOldTweets3 --querysearch "museum" --near "55.75, 37.61" --within 40km --maxtweets 10 
"""

import os, sys, re, getopt
import traceback

if sys.version_info[0] < 3:
    raise Exception("Python 2.x is not supported. Please upgrade to 3.x")

import GetOldTweets3 as got

def main(argv):
    if len(argv) == 0:
        print('You must pass some parameters. Use \"-h\" to help.')
        return

    if len(argv) == 1 and argv[0] == '-h':
        print(__doc__)
        return

    try:
        opts, args = getopt.getopt(argv, "", ("querysearch=",
                                              "username=",
                                              "usernames-from-file=",
                                              "since=",
                                              "until=",
                                              "near=",
                                              "within=",
                                              "toptweets",
                                              "maxtweets=",
                                              "lang=",
                                              "output=",
                                              "debug"))

        tweetCriteria = got.manager.TweetCriteria()
        outputFileName = "output_got.csv"

        debug = False
        usernames = set()
        username_files = set()
        for opt, arg in opts:
            if opt == '--querysearch':
                tweetCriteria.querySearch = arg

            elif opt == '--username':
                usernames_ = [u.lstrip('@') for u in re.split(r'[\s,]+', arg) if u]
                usernames_ = [u.lower() for u in usernames_ if u]
                usernames |= set(usernames_)

            elif opt == '--usernames-from-file':
                username_files.add(arg)

            elif opt == '--since':
                tweetCriteria.since = arg

            elif opt == '--until':
                tweetCriteria.until = arg

            elif opt == '--near':
                geocode = arg.split(',')
                try:
                    if len(geocode) != 2:
                        raise
                    lat, lon = geocode[0].strip(), geocode[1].strip()
                    if lat[-1].lower() == 'n':
                        lat = float(lat[:-1])
                    elif lat[-1].lower() == 's':
                        lat = -float(lat[:-1])
                    else:
                        lat = float(lat)

                    if lon[-1].lower() == 'e':
                        lon = float(lon[:-1])
                    elif lon[-1].lower() == 'w':
                        lon = -float(lon[:-1])
                    else:
                        lon = float(lon)
                    if lat < -180 or lat > 180:
                        raise
                    if lon < -90 or lon > 90:
                        raise
                    tweetCriteria.lat = lat
                    tweetCriteria.lon = lon
                except:
                    tweetCriteria.near = arg

            elif opt == '--within':
                tweetCriteria.within = arg

            elif opt == '--toptweets':
                tweetCriteria.topTweets = True

            elif opt == '--maxtweets':
                tweetCriteria.maxTweets = int(arg)

            elif opt == '--lang':
                tweetCriteria.lang = arg

            elif opt == '--output':
                outputFileName = arg

            elif opt == '--debug':
                debug = True

        if debug:
            print(' '.join(sys.argv))
            print("GetOldTweets3", got.__version__)

        if username_files:
            for uf in username_files:
                if not os.path.isfile(uf):
                    raise Exception("File not found: %s"%uf)
                with open(uf) as f:
                    data = f.read()
                    data = re.sub('(?m)#.*?$', '', data)  # remove comments
                    usernames_ = [u.lstrip('@') for u in re.split(r'[\s,]+', data) if u]
                    usernames_ = [u.lower() for u in usernames_ if u]
                    usernames |= set(usernames_)
                    print("Found %i usernames in %s" % (len(usernames_), uf))

        if usernames:
            if len(usernames) > 1:
                tweetCriteria.username = usernames
                if len(usernames)>20 and tweetCriteria.maxTweets > 0:
                    maxtweets_ = (len(usernames) // 20 + (len(usernames)%20>0)) * tweetCriteria.maxTweets
                    print("Warning: due to multiple username batches `maxtweets' set to %i" % maxtweets_)
            else:
                tweetCriteria.username = usernames.pop()

        outputFile = open(outputFileName, "w+", encoding="utf8")
        outputFile.write('date,username,to,replies,retweets,favorites,text,geo,mentions,hashtags,id,permalink\n')

        cnt = 0
        def receiveBuffer(tweets):
            nonlocal cnt

            for t in tweets:
                data = [t.date.strftime("%Y-%m-%d %H:%M:%S"),
                    t.username,
                    t.to or '',
                    t.replies,
                    t.retweets,
                    t.favorites,
                    '"'+t.text.replace('"','""')+'"',
                    t.geo,
                    t.mentions,
                    t.hashtags,
                    t.id,
                    t.permalink]
                data[:] = [i if isinstance(i, str) else str(i) for i in data]
                outputFile.write(','.join(data) + '\n')

            outputFile.flush()
            cnt += len(tweets)

            if sys.stdout.isatty():
                print("\rSaved %i"%cnt, end='', flush=True)
            else:
                print(cnt, end=' ', flush=True)

        print("Downloading tweets...")
        got.manager.TweetManager.getTweets(tweetCriteria, receiveBuffer, debug=debug)

    except getopt.GetoptError as err:
        print('Arguments parser error, try -h')
        print('\t' + str(err))

    except KeyboardInterrupt:
        print("\r\nInterrupted.\r\n")

    except Exception as err:
        print(traceback.format_exc())
        print(str(err))

    finally:
        if "outputFile" in locals():
            outputFile.close()
            print()
            print('Done. Output file generated "%s".' % outputFileName)

if __name__ == '__main__':
    main(sys.argv[1:])


In [ ]:
import time
import pandas as pd
import tweepy
from tweepy import OAuthHandler
import GetOldTweets3 as got

In [ ]:
#pip install tweepy

In [ ]:
# authorize the app to access Twitter Streaming API
access_token = ""
access_token_secret = ""
consumer_key = ""
consumer_secret = ""

In [ ]:
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

In [ ]:
#list of column names
COLS = ['id', 'created_at','original_text', 'screen_name', 'author_id',
        'replies', 'retweet_count', 'to','hashtags',
        'user_mentions', 'urls']

In [ ]:

path =r"/Users//Desktop/Senti Project"

### Fetching the tweets for the election year

2015 ELECTION

In [ ]:
politicians=['MBuhari','GEJonathan' ]

In [ ]:
#Function to fetch the tweet object, grab the information and write into a .csv file
for screenname in politicians:
    entry = []
    tweetCriteria = got.manager.TweetCriteria().setUsername(screenname)\
                                               .setSince("2014-12-29")\
                                               .setUntil("2015-03-29")\
                                               .setMaxTweets(1000)\
                                               .setEmoji("unicode")
    tweet = got.manager.TweetManager.getTweets(tweetCriteria)
    for twt in tweet:
        entry.append([twt.id,twt.formatted_date,twt.text,twt.username,twt.author_id,twt.replies,twt.retweets,twt.to,twt.hashtags,twt.mentions,twt.urls])
    print(len(entry),screenname)
    df = pd.DataFrame(entry,columns=COLS)
    # print(df.head())
    df.to_csv(path+'\\'+screenname+"_tweets.csv", mode='a+', columns=COLS, index=False,
                                      encoding="utf-8")
    time.sleep(3)

In [ ]:
hashs = ['buhari','jonathan','pdp','apc']

In [ ]:
#Function to fetch the tweet object, grab the information and write into a .csv file
for screenname in hashs:
    entry = []
    tweetCriteria = got.manager.TweetCriteria().setQuerySearch(screenname)\
                                               .setSince("2014-12-29")\
                                               .setUntil("2015-03-29")\
                                               .setMaxTweets(20000)\
                                               .setEmoji("unicode")
    tweet = got.manager.TweetManager.getTweets(tweetCriteria)
    for twt in tweet:
        entry.append([twt.id,twt.formatted_date,twt.text,twt.username,twt.author_id,twt.replies,twt.retweets,twt.to,twt.hashtags,twt.mentions,twt.urls])
    print(len(entry),screenname)
    df = pd.DataFrame(entry,columns=COLS)
    # print(df.head())
    df.to_csv(path+'\\'+screenname+"_tweets.csv", mode='a+', columns=COLS, index=False,
                                      encoding="utf-8")
    time.sleep(3)